# Problema 1:

Determine los desplazamientos de los bordes libres y los esfuerzos en función de la posición para la
placa empotrada mostrada en la figura. Tome las dos particiones en elementos mostradas debajo y
compare los resultados. Asuma que E = 30 x 106
 psi y n = 0.30.

Partición 1

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import copy

E = 30e6     # psi
v = 0.3      # Poisson
t = 1        # in (espesor)
h = 10       # in (altura)
l = 20       # in (largo)
T = 10000     # psi (tensión aplicada)
Fext = T / 2  # lb (fuerzas)

MN1 = [[0,0], [0,h], [l,h], [l,0], [l/2, h/2]]
MN2 = [[0,0], [0,h], [l,h], [l,0], [l/2, h/2]]

MC1 = [[0,2,1], [0,3,2]]
MC2 = [[4,0,1], [4,0,3], [4,2,3], [4,2,3]]
CC = [[2, 0], [2, 0], [2, 0], [2, 0], [1, Fext], [1, 0], [1, Fext], [1,0]]

GL = 2     # Grados de libertad de un nodo




def MEF(GL, MN, MC, CC, E, v, t, h, l):
    nod = len(MN)    # cantidad de nodos
    e = len(MC)      # cantidad de elementos
    K = np.zeros([nod*GL, nod*GL])
    Esf= []
    Bs=[]
    for a in range(e):
        A = (h*l)/2
        B = np.zeros([3,6])
        c = []    #coordenadas de los nodos del elemento
        for i in MC[a]: 
            c.append(MN[i])
            b=[0,0,0]
            g=[0,0,0]
        for d in range(3): 
            b[d] = c[d-2][1]- c[d-1][1]
            g[d] = c[d-1][0]- c[d-2][0]
        for j in range(3):
            B0=[[b[j],0],[0, g[j]], [g[j], b[j]]]
            B[0][j*2]=b[j]
            B[1][2*j+1]=g[j]
            B[2][j*2]=g[j]
            B[2][j*2+1]=b[j]
        B = B/(2*A)
        Bs.append(B)
        D = (E/(1-(v**2)))*np.array([[1,v,0],[v,1,0],[0,0,(1-v)/2]])
        k = t*abs(A)*np.dot(B.transpose(), np.dot(D,B) )
        K0 = np.zeros([nod*2, nod*2])
        for i in range(3):
            nx=MC[a][i]
            for j in range(3):
                ny=MC[a][j]
                K0[np.ix_([nx*2,nx*2+1], [ny*2,ny*2+1])]+=k[i*2:i*2+2, j*2:j*2+2]
        K+=K0



    #indices de Fuerzas y desplazamientos conocidxs:
    d = np.zeros(nod*GL)
    f = np.zeros(nod*GL)
    fc=[]
    dc=[]
    for i in range(len(CC)):
        if CC[i][0]==1:
            fc.append([i,CC[i][1]])
        else: 
            dc.append([i,CC[i][1]])
            d[i]=CC[i][1]

    fc=np.array(fc).astype(int)
    dc=np.array(dc)

    dd = np.linalg.solve(K[np.ix_(fc[:,0], fc[:,0])], fc[:,1])

    for i in range(len(fc)):
        d[fc[i,0]]=dd[i]


    F=K.dot(d)
    B=np.array(B)
    D=np.array(D)

    for i in range(e):
        j=np.array(MC[i])*2 
        Esf.append( np.dot( np.dot(D,Bs[i]), d[[j[0],j[0]+1,j[1],j[1]+1,j[2],j[2]+1]] ) ) 
    
    return d, Esf, F



In [8]:
# Particion 1
MEF(GL, MN1, MC1, CC, E, v, t, h, l)

(array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        6.09580998e-04, 4.16333066e-06, 6.63704297e-04, 1.04083267e-04,
        0.00000000e+00, 0.00000000e+00]),
 [array([1004.80384307,  301.44115292,    2.40192154]),
  array([995.19615693,  -1.20096077,  -2.40192154])],
 array([-5.00000000e+03, -3.00240192e+03, -5.00000000e+03,  3.00240192e+03,
         5.00000000e+03,  1.13686838e-13,  5.00000000e+03, -9.09494702e-13,
         0.00000000e+00,  0.00000000e+00]))

In [9]:
# Particion 2
MEF(GL, MN2, MC2, CC, E, v, t, h, l)

(array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         6.72541474e-04, -1.82140761e-04,  4.12746867e-04,  1.11553207e-05,
         0.00000000e+00,  0.00000000e+00]),
 [array([0., 0., 0.]),
  array([ 334.65962212,   83.66490553, -234.90531168]),
  array([-798.88243863,   50.27939034, -100.55878068]),
  array([-798.88243863,   50.27939034, -100.55878068])],
 array([ 3.37877503e+02,  1.68938752e+02,  0.00000000e+00,  0.00000000e+00,
         5.00000000e+03,  1.19371180e-12,  5.00000000e+03,  0.00000000e+00,
        -1.03378775e+04, -1.68938752e+02]))